# OpenAI Streaming - Traced With LastMile

Four use cases:

1. Text - Non-streaming
2. [this one] Text - Streaming
3. Tool Calls - Non-streaming
4. Tool Calls - Streaming

In [1]:
# !pip install scipy --quiet
# !pip install tenacity --quiet
# !pip install tiktoken --quiet
# !pip install termcolor --quiet
# !pip install openai --quiet
# !pip install "tracing-auto-instrumentation[openai]"

# Create ~/.env file with this line: OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
import openai

from lastmile_eval.rag.debugger.api.tracing import LastMileTracer

from tracing_auto_instrumentation.openai import wrap_openai
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="OpenAI Text Calling w. Streaming",
)
client = openai.OpenAI(api_key=OPENAI_API_KEY)
client = wrap_openai(client, tracer)

In [3]:
def run_my_existing_openai_app(user_message: str, stream: bool = True):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 10,
        "temperature": 1,
        "stream": stream,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = client.chat.completions.create(**completion_params)
    print("Chat Completion Response: ")
    if stream:
        for chunk in response:
            print(f"{chunk=}")
    else:
        print(f"{response=}")

In [4]:
# # Run your code as usual
# stream = False
# run_my_existing_openai_app("Tell me a joke about apples", stream=stream)

## Time to test this with async calls

In [5]:
client = openai.AsyncOpenAI(api_key=OPENAI_API_KEY)
client = wrap_openai(client, tracer)

async def run_my_existing_openai_app_async(user_message: str, stream: bool = True):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": stream,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = await client.chat.completions.create(**completion_params)
    print("Chat Completion Response: ")
    if stream:
        async for chunk in response:
            print(f"{chunk=}")
    else:
        print(f"{response=}")
    return response

2024-05-30 19:07:57,805 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-30 19:07:57,806 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'


In [ ]:
stream = False
response = await run_my_existing_openai_app_async("Tell me a joke about apples", stream=stream)